In [36]:
project_name = 'full_fine_tuning_5epochs_final1'

datas=( 'stanford_cars', 'caltech101', 'fgvc-aircraft-2013b', 'dtd', 'flowers-102', 'oxford-iiit-pet'  ) #'imagenette2' 'eurosat' 

losses=( 'TRADES_v2', 'CLASSIC_AT' ) #  

backbones=(
  'convnext_tiny', 'robust_convnext_tiny', 'convnext_tiny.fb_in22k', 
  'deit_small_patch16_224.fb_in1k', 'robust_deit_small_patch16_224',
  'convnext_base', 'convnext_base.fb_in22k', 'robust_convnext_base', 
  'convnext_base.clip_laion2b', 'convnext_base.clip_laion2b_augreg',
  'vit_base_patch16_224.augreg_in1k', 'vit_base_patch16_224.augreg_in21k',
  'vit_base_patch16_224.dino', 'vit_base_patch16_224.mae', 'vit_base_patch16_224.orig_in21k',
  'vit_base_patch16_224.sam_in1k', 'vit_base_patch16_224_miil.in21k'  ) 

import pickle
import numpy as np

final_data = []

for backbone in backbones:
    for loss in losses:
        for data in datas:

            try:

                name ='{}_{}_{}'.format(backbone, data, loss)
                with open('./results/results_{}_{}_{}_{}.pkl'.format(project_name, backbone, data, loss), 'rb') as f:
                    saved_data = pickle.load(f)
                    result = saved_data[name]["statistics"]
                    result['backbone'] = backbone
                    result['dataset'] = data
                    result['loss_function'] = loss    

                    print(name, saved_data[name]["statistics"])
            except:
                result = {}
                result['backbone'] = backbone
                result['dataset'] = data
                result['loss_function'] = loss   
                result['clean_acc'] = np.nan
                result['robust_acc'] = np.nan 
                # print('not available')

            final_data.append(result)


convnext_tiny_stanford_cars_TRADES_v2 {'clean_acc': 0.3606666666666667, 'robust_acc': 0.004, 'backbone': 'convnext_tiny', 'dataset': 'stanford_cars', 'loss_function': 'TRADES_v2'}
convnext_tiny_caltech101_TRADES_v2 {'clean_acc': 0.9466666666666667, 'robust_acc': 0.8073333333333333, 'backbone': 'convnext_tiny', 'dataset': 'caltech101', 'loss_function': 'TRADES_v2'}
convnext_tiny_fgvc-aircraft-2013b_TRADES_v2 {'clean_acc': 0.14466666666666667, 'robust_acc': 0.0006666666666666666, 'backbone': 'convnext_tiny', 'dataset': 'fgvc-aircraft-2013b', 'loss_function': 'TRADES_v2'}
convnext_tiny_dtd_TRADES_v2 {'clean_acc': 0.49, 'robust_acc': 0.09083333333333334, 'backbone': 'convnext_tiny', 'dataset': 'dtd', 'loss_function': 'TRADES_v2'}
convnext_tiny_flowers-102_TRADES_v2 {'clean_acc': 0.8506666666666667, 'robust_acc': 0.16233333333333333, 'backbone': 'convnext_tiny', 'dataset': 'flowers-102', 'loss_function': 'TRADES_v2'}
convnext_tiny_oxford-iiit-pet_TRADES_v2 {'clean_acc': 0.852, 'robust_acc':

In [40]:
import pandas as pd

df = pd.DataFrame(final_data)
df.to_csv('results.csv')
df

,clean_acc,robust_acc,backbone,dataset,loss_function
0,0.360667,0.004000,convnext_tiny,stanford_cars,TRADES_v2
1,0.946667,0.807333,convnext_tiny,caltech101,TRADES_v2
2,0.144667,0.000667,convnext_tiny,fgvc-aircraft-2013b,TRADES_v2
3,0.490000,0.090833,convnext_tiny,dtd,TRADES_v2
4,0.850667,0.162333,convnext_tiny,flowers-102,TRADES_v2
...,...,...,...,...,...
199,0.326667,0.252667,vit_base_patch16_224_miil.in21k,caltech101,CLASSIC_AT
200,0.022667,0.002000,vit_base_patch16_224_miil.in21k,fgvc-aircraft-2013b,CLASSIC_AT
201,0.328000,0.189333,vit_base_patch16_224_miil.in21k,dtd,CLASSIC_AT
202,0.284667,0.203333,vit_base_patch16_224_miil.in21k,flowers-102,CLASSIC_AT


In [39]:
data_updated = df.copy()

# Combine clean_acc and robust_acc as independent metrics for ranking
data_updated['total_acc'] = data_updated['clean_acc'] + data_updated['robust_acc']  # Sum of scores per dataset

# Aggregate total_acc across all datasets for each backbone + loss combination
aggregated_data_no_avg = data_updated.groupby(['backbone', 'loss_function']).agg(
    total_sum=('total_acc', 'sum'),  # Total sum of all scores across datasets
    total_geom_mean=('total_acc', lambda x: np.prod(x) ** (1 / len(x))),  # Geometric mean
).reset_index()

# Rank based on these combined scores
aggregated_data_no_avg['rank_total_sum'] = aggregated_data_no_avg['total_sum'].rank(ascending=False)
aggregated_data_no_avg['rank_total_geom_mean'] = aggregated_data_no_avg['total_geom_mean'].rank(ascending=False)

# Sort by sum ranking for presentation
ranked_aggregated_data_no_avg = aggregated_data_no_avg.sort_values(by='rank_total_sum')

ranked_aggregated_data_no_avg

# tools.display_dataframe_to_user(name="Ranked Backbone and Loss Combinations Without Averaging", dataframe=ranked_aggregated_data_no_avg)




,backbone,loss_function,total_sum,total_geom_mean,rank_total_sum,rank_total_geom_mean
7,convnext_base.fb_in22k,TRADES_v2,7.713333,1.184050,1.000000,1.000000
1,convnext_base,TRADES_v2,7.380667,1.062435,2.000000,3.000000
3,convnext_base.clip_laion2b,TRADES_v2,7.035333,1.106352,3.000000,2.000000
11,convnext_tiny.fb_in22k,TRADES_v2,6.682667,1.008759,4.000000,5.000000
4,convnext_base.clip_laion2b_augreg,CLASSIC_AT,6.151167,0.882778,5.000000,8.000000
29,vit_base_patch16_224.orig_in21k,TRADES_v2,6.116667,0.833713,6.000000,12.000000
17,robust_convnext_tiny,TRADES_v2,5.876000,0.839919,7.000000,11.000000
5,convnext_base.clip_laion2b_augreg,TRADES_v2,5.870000,1.050918,8.000000,4.000000
16,robust_convnext_tiny,CLASSIC_AT,5.579333,0.677827,9.000000,18.000000
2,convnext_base.clip_laion2b,CLASSIC_AT,5.528000,0.980741,10.000000,7.000000


In [41]:


# Pivot the table to group by 'backbone' and 'loss_function' in rows and create a hierarchical structure with 'dataset' as column index
grouped_df = df.pivot_table(
    index=['backbone', 'loss_function'], 
    columns='dataset', 
    values=[ 'clean_acc',  ], #'robust_acc'
    dropna=False
)

grouped_df #.to_csv('results_acc.csv' )

clean_acc           \
dataset                                         caltech101      dtd   
backbone                          loss_function                       
convnext_base                     CLASSIC_AT      0.982000 0.680000   
                                  TRADES_v2       0.977333 0.720667   
convnext_base.clip_laion2b        CLASSIC_AT      0.948000 0.562667   
                                  TRADES_v2       0.955333 0.600000   
convnext_base.clip_laion2b_augreg CLASSIC_AT      0.916667 0.566000   
                                  TRADES_v2       0.948000      NaN   
convnext_base.fb_in22k            CLASSIC_AT      0.983333 0.714000   
                                  TRADES_v2       0.982000 0.677333   
convnext_tiny                     CLASSIC_AT      0.966000 0.508000   
                                  TRADES_v2       0.946667 0.490000   
convnext_tiny.fb_in22k            CLASSIC_AT      0.970667 0.647111   
                                  TRADES_v2       0.967333 0.594667   
deit_small_patch16_224.fb_in1k    CLASSIC_AT      0.949333 0.586000   
                                  TRADES_v2       0.947333      NaN   
robust_convnext_base              CLASSIC_AT      0.952000 0.495333   
                                  TRADES_v2       0.966000 0.458667   
robust_convnext_tiny              CLASSIC_AT      0.962667 0.510000   
                                  TRADES_v2       0.955333 0.460000   
robust_deit_small_patch16_224     CLASSIC_AT      0.962000 0.490667   
                                  TRADES_v2       0.959333      NaN   
vit_base_patch16_224.augreg_in1k  CLASSIC_AT      0.912667 0.530667   
                                  TRADES_v2       0.926667 0.534667   
vit_base_patch16_224.augreg_in21k CLASSIC_AT      0.924000 0.530667   
                                  TRADES_v2       0.960000 0.416667   
vit_base_patch16_224.dino         CLASSIC_AT      0.956667 0.584000   
                                  TRADES_v2       0.931333      NaN   
vit_base_patch16_224.mae          CLASSIC_AT      0.806667 0.318000   
                                  TRADES_v2       0.860000 0.333333   
vit_base_patch16_224.orig_in21k   CLASSIC_AT      0.877333 0.589333   
                                  TRADES_v2       0.942000 0.684667   
vit_base_patch16_224.sam_in1k     CLASSIC_AT      0.920667 0.539333   
                                  TRADES_v2            NaN 0.608667   
vit_base_patch16_224_miil.in21k   CLASSIC_AT      0.326667 0.328000   
                                  TRADES_v2       0.888667      NaN   

                                                                     \
dataset                                         fgvc-aircraft-2013b   
backbone                          loss_function                       
convnext_base                     CLASSIC_AT               0.117333   
                                  TRADES_v2                0.249333   
convnext_base.clip_laion2b        CLASSIC_AT               0.304667   
                                  TRADES_v2                0.465333   
convnext_base.clip_laion2b_augreg CLASSIC_AT               0.290667   
                                  TRADES_v2                0.422000   
convnext_base.fb_in22k            CLASSIC_AT               0.214000   
                                  TRADES_v2                0.425333   
convnext_tiny                     CLASSIC_AT               0.063333   
                                  TRADES_v2                0.144667   
convnext_tiny.fb_in22k            CLASSIC_AT               0.185333   
                                  TRADES_v2                0.414667   
deit_small_patch16_224.fb_in1k    CLASSIC_AT               0.076000   
                                  TRADES_v2                0.114667   
robust_convnext_base              CLASSIC_AT               0.210000   
                                  TRADES_v2                0.075333   
robust_convnext_tiny              CLASSIC_AT               0.090000   
 

In [42]:


# Pivot the table to group by 'backbone' and 'loss_function' in rows and create a hierarchical structure with 'dataset' as column index
grouped_df = df.pivot_table(
    index=['backbone', 'loss_function'], 
    columns='dataset', 
    values=['robust_acc', ], #
    dropna=False
)

grouped_df

robust_acc           \
dataset                                         caltech101      dtd   
backbone                          loss_function                       
convnext_base                     CLASSIC_AT      0.924667 0.460667   
                                  TRADES_v2       0.922000 0.355333   
convnext_base.clip_laion2b        CLASSIC_AT      0.876667 0.331333   
                                  TRADES_v2       0.833333 0.245333   
convnext_base.clip_laion2b_augreg CLASSIC_AT      0.870000 0.346000   
                                  TRADES_v2       0.836000      NaN   
convnext_base.fb_in22k            CLASSIC_AT      0.928000 0.251833   
                                  TRADES_v2       0.924667 0.339333   
convnext_tiny                     CLASSIC_AT      0.845333 0.061500   
                                  TRADES_v2       0.807333 0.090833   
convnext_tiny.fb_in22k            CLASSIC_AT      0.891333 0.296889   
                                  TRADES_v2       0.892000 0.224000   
deit_small_patch16_224.fb_in1k    CLASSIC_AT      0.841333 0.312667   
                                  TRADES_v2       0.818667      NaN   
robust_convnext_base              CLASSIC_AT      0.929333 0.411333   
                                  TRADES_v2       0.931333 0.341333   
robust_convnext_tiny              CLASSIC_AT      0.921333 0.367333   
                                  TRADES_v2       0.899333 0.260667   
robust_deit_small_patch16_224     CLASSIC_AT      0.904667 0.338667   
                                  TRADES_v2       0.890000      NaN   
vit_base_patch16_224.augreg_in1k  CLASSIC_AT      0.828667 0.289333   
                                  TRADES_v2       0.814000 0.239333   
vit_base_patch16_224.augreg_in21k CLASSIC_AT      0.788667 0.277000   
                                  TRADES_v2       0.848000 0.046833   
vit_base_patch16_224.dino         CLASSIC_AT      0.838667 0.296667   
                                  TRADES_v2       0.824000      NaN   
vit_base_patch16_224.mae          CLASSIC_AT      0.694667 0.192667   
                                  TRADES_v2       0.633333 0.142000   
vit_base_patch16_224.orig_in21k   CLASSIC_AT      0.804000 0.335333   
                                  TRADES_v2       0.818000 0.215333   
vit_base_patch16_224.sam_in1k     CLASSIC_AT      0.818000 0.322667   
                                  TRADES_v2            NaN 0.281333   
vit_base_patch16_224_miil.in21k   CLASSIC_AT      0.252667 0.189333   
                                  TRADES_v2       0.536667      NaN   

                                                                     \
dataset                                         fgvc-aircraft-2013b   
backbone                          loss_function                       
convnext_base                     CLASSIC_AT               0.006500   
                                  TRADES_v2                0.088000   
convnext_base.clip_laion2b        CLASSIC_AT               0.100000   
                                  TRADES_v2                0.179333   
convnext_base.clip_laion2b_augreg CLASSIC_AT               0.106000   
                                  TRADES_v2                0.014667   
convnext_base.fb_in22k            CLASSIC_AT               0.004667   
                                  TRADES_v2                0.134667   
convnext_tiny                     CLASSIC_AT               0.000000   
                                  TRADES_v2                0.000667   
convnext_tiny.fb_in22k            CLASSIC_AT               0.003333   
                                  TRADES_v2                0.083333   
deit_small_patch16_224.fb_in1k    CLASSIC_AT               0.008000   
                                  TRADES_v2                0.015333   
robust_convnext_base              CLASSIC_AT               0.146667   
                                  TRADES_v2                0.040000   
robust_convnext_tiny              CLASSIC_AT               0.050667   


In [22]:
import pandas as pd
import numpy as np
import plotly.express as px


df_new_cleaned = df #.drop(columns=['Unnamed: 0'])
name =  'CLASSIC_AT' #'TRADES_v2' #
df_new_cleaned = df_new_cleaned[ df_new_cleaned.loss_function != name ]

# Assuming df_new_cleaned already exists and contains all necessary columns
# Group by backbone and loss function, compute the mean and standard deviation for clean and robust accuracy
aggregated_metrics = df_new_cleaned.groupby(['backbone', 'loss_function']).agg(
    Mean_Clean_Acc=('clean_acc', 'mean'),
    Std_Clean_Acc=('clean_acc', 'std'),
    Mean_Robust_Acc=('robust_acc', 'mean'),
    Std_Robust_Acc=('robust_acc', 'std')
).reset_index()

# Classify architectures into categories
aggregated_metrics['Architecture'] = aggregated_metrics['backbone'].apply(
    lambda name: 'ConvNext' if 'convnext' in name.lower() else ('ViT/DeiT' if 'deit' in name.lower() or 'vit' in name.lower() else 'Other')
)

# Assign marker shapes based on loss function
aggregated_metrics['Marker_Shape'] = aggregated_metrics['loss_function'].apply(
    lambda loss: 'circle' if loss == 'TRADES_v2' else ('square' if loss == 'CLASSIC_AT' else 'diamond')
)

# Identify Pareto front
x = aggregated_metrics['Mean_Clean_Acc']
y = aggregated_metrics['Mean_Robust_Acc']
is_pareto = np.array([
    not any((y[j] >= y[i] and x[j] >= x[i]) for j in range(len(x)) if i != j)
    for i in range(len(x))
])
pareto_points = aggregated_metrics[is_pareto]

# Add a mapping for loss functions to valid Plotly symbols
symbol_mapping = {
    'TRADES_v2': 'circle',
    'CLASSIC_AT': 'square'
}

# Update the scatter plot with proper legend entries for loss functions
fig = px.scatter(
    aggregated_metrics,
    x='Mean_Clean_Acc',
    y='Mean_Robust_Acc',
    color='Architecture',         # Color for architecture
    symbol='loss_function',       # Symbol for loss function
    # title='Backbone Performance: Clean vs. Robust Accuracy',
    labels={
        'Mean_Clean_Acc': 'Mean Clean Accuracy',
        'Mean_Robust_Acc': 'Mean Robust Accuracy',
        'loss_function': 'Loss Function'  # Custom label for legend
    },
    color_discrete_map={
        'ConvNext': '#1f77b4',  # Blue for ConvNext
        'ViT/DeiT': '#ff7f0e',  # Orange for ViT/DeiT
        'Other': '#2ca02c'      # Green for Other architectures
    },
    symbol_map=symbol_mapping    # Apply symbol mapping
)

# Add static backbone names as labels on the figure
for i, row in aggregated_metrics.iterrows():
    fig.add_annotation(
        x=row['Mean_Clean_Acc'] - 0.01,
        y=row['Mean_Robust_Acc'] + 0.01,  # Offset text slightly above the marker
        text=row['backbone'],
        showarrow=False,
        font=dict(size=10),
        align='center'
    )

# Add Pareto front markers with larger size and same shape/color as the original
for idx, d in enumerate(pareto_points.iterrows()):
    _, row = d
    fig.add_scatter(
        x=[row['Mean_Clean_Acc']],
        y=[row['Mean_Robust_Acc']],
        mode='markers',
        showlegend=False,  # Avoid duplicating legends
        marker=dict(
            size=13,  # Larger marker size
            symbol=symbol_mapping[row['loss_function']],  # Use the mapped symbol
            color=aggregated_metrics.loc[aggregated_metrics['backbone'] == row['backbone'], 'Architecture'].map({
                'ConvNext': '#1f77b4',  # Blue for ConvNext
                'ViT/DeiT': '#ff7f0e',  # Orange for ViT/DeiT
                'Other': '#2ca02c'     # Green for Other architectures
            }).values[0]  # Map the color dynamically
        ),
        name=f"Pareto"
    )


# Add a diagonal line (x = y)
fig.add_shape(
    type="line",
    x0=0, y0=0,  # Start of line
    x1=1, y1=1,  # End of line
    xref="paper", yref="paper",  # Use normalized coordinates (0 to 1 in plot space)
    line=dict(color="red", width=2, dash="dot")  # Line style
)

# Adjust layout for better visual clarity
fig.update_layout(
    legend_title=dict(text='Backbone and Loss Function'),
    width=600,
    height=600,
    font=dict(size=12),
    #title_font=dict(size=14),
    margin=dict(l=10, r=45, t=10, b=10),
    plot_bgcolor='rgba(0,0,0,0)',
    showlegend=False,
    xaxis=dict(showgrid=True, gridcolor='lightgrey', zeroline=False),
    yaxis=dict(showgrid=True, gridcolor='lightgrey', zeroline=False),
    yaxis_range=[0,0.7],
    xaxis_range=[0,0.85]
)


fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85 ],
        ticktext=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85 ]
    ),
    yaxis=dict(
        tickmode="array",
        tickvals=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7 ],
        ticktext=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7 ]
    )
)

fig.show()
fig.write_image( "./figures/{}_{}.png".format(project_name,name) )

In [23]:
import comet_ml
import os

os.environ["COMET_API_KEY"] = "I5AiXfuD0TVuSz5UOtujrUM9i"

PROJECT_NAME="full-fine-tuning-5epochs-final1"
WORKSPACE_NAME="maxheuillet"

# Get all Experiemnts in workspace
api = comet_ml.API()

# all_experiments_in_workspace = api.get_experiments(WORKSPACE_NAME)
all_experiments_in_project = api.get_experiments(WORKSPACE_NAME, PROJECT_NAME)


backbones = [
  "convnext_tiny", "robust_convnext_tiny", "convnext_tiny.fb_in22k", 
  "deit_small_patch16_224.fb_in1k", "robust_deit_small_patch16_224",
  "convnext_base", "convnext_base.fb_in22k", "robust_convnext_base",
  "convnext_base.clip_laion2b", "convnext_base.clip_laion2b_augreg",
  "vit_base_patch16_224.augreg_in1k", "vit_base_patch16_224.augreg_in21k",
  "vit_base_patch16_224.dino", "vit_base_patch16_224.mae", 
  "vit_base_patch16_224.orig_in21k", "vit_base_patch16_224.sam_in1k",
  "vit_base_patch16_224_miil.in21k", ]

datas = [ "stanford_cars", "caltech101", "fgvc-aircraft-2013b", 
          "dtd", "flowers-102", "oxford-iiit-pet", ]

losses = [ "TRADES_v2", "CLASSIC_AT", ]

# 1) Build a dictionary
experiment_map = {}
for b in backbones:
    for d in datas:
        for l in losses:
            exp_name = f"{b}_{d}_{l}"
            experiment_map[exp_name] = (b, d, l)

In [24]:
import tqdm.notebook as tqdm
import pandas as pd

# data = all_experiments_in_project[0].get_metrics()
# import pandas as pd
# data = pd.DataFrame(data)
# set(data.metricName)

dataframe = []

for i in range(len(all_experiments_in_project)) :

    # 2) Suppose we have an experiment name to parse
    experiment_name = all_experiments_in_project[i].get_name()

    # 3) Look it up
    if experiment_name in experiment_map:
        backbone, dataset, loss = experiment_map[experiment_name]
        print("Backbone:", backbone)
        print("Dataset:", dataset)
        print("Loss:", loss)
    else:
        print("Experiment name not found in known combinations!")

    data = all_experiments_in_project[i].get_metrics()

    data = pd.DataFrame(data)

    df_filtered = data[ data.metricName == 'zero_adv_val' ]
    df_filtered = df_filtered.sort_values("step")
    zero_adv_val = df_filtered["metricValue"].tolist()
    zero_adv_val = [float(x) for x in zero_adv_val]

    df_filtered = data[ data.metricName == 'dormant_adv_val' ]
    df_filtered = df_filtered.sort_values("step")
    dormant_adv_val = df_filtered["metricValue"].tolist()
    dormant_adv_val = [float(x) for x in dormant_adv_val]

    df_filtered = data[ data.metricName == 'overactive_adv_val' ]
    df_filtered = df_filtered.sort_values("step")
    overactive_adv_val = df_filtered["metricValue"].tolist()
    overactive_adv_val = [float(x) for x in overactive_adv_val]


    df_filtered = data[ data.metricName == 'zero_adv_train' ]
    df_filtered = df_filtered.sort_values("step")
    zero_adv_train = df_filtered["metricValue"].tolist()
    zero_adv_train = [float(x) for x in zero_adv_train]

    df_filtered = data[ data.metricName == 'dormant_adv_train' ]
    df_filtered = df_filtered.sort_values("step")
    dormant_adv_train = df_filtered["metricValue"].tolist()
    dormant_adv_train = [float(x) for x in dormant_adv_train]

    df_filtered = data[ data.metricName == 'overactive_adv_train' ]
    df_filtered = df_filtered.sort_values("step")
    overactive_adv_train = df_filtered["metricValue"].tolist()
    overactive_adv_train = [float(x) for x in overactive_adv_train]

    df_filtered = data[ data.metricName == 'gradient_norm' ]
    df_filtered = df_filtered.sort_values("step")
    gradient_norm = df_filtered["metricValue"].tolist()
    gradient_norm = [float(x) for x in gradient_norm]

    if len(zero_adv_train) != 0:

        result = {'backbone':backbone, 'loss_function':loss, 'dataset':dataset,
                
            'zero_adv_train0':zero_adv_train[0], 
            'dormant_adv_train0':dormant_adv_train[0], 
            'overactive_adv_train0':overactive_adv_train[0], 
                
            'zero_adv_train5':zero_adv_train[-1], 
            'dormant_adv_train5':dormant_adv_train[-1], 
            'overactive_adv_train5':overactive_adv_train[-1],

            'zero_adv_val0':zero_adv_val[0], 
            'dormant_adv_val0':dormant_adv_val[0], 
            'overactive_adv_val0':overactive_adv_val[0], 
                
            'zero_adv_val5':zero_adv_val[-1], 
            'dormant_adv_val5':dormant_adv_val[-1], 
            'overactive_adv_val5':overactive_adv_val[-1], }
        
    else:

        pass

        # result = {'backbone':backbone, 'loss_function':loss, 'dataset':dataset,
                
        #         'zero_adv_train0':np.nan, 
        #         'dormant_adv_train0':np.nan, 
        #         'overactive_adv_train0':np.nan, 
                
        #         'zero_adv_train5':np.nan, 
        #         'dormant_adv_train5':np.nan, 
        #         'overactive_adv_train5':np.nan,  

        #         'zero_adv_val0':np.nan, 
        #         'dormant_adv_val0':np.nan, 
        #         'overactive_adv_val0':np.nan, 
                    
        #         'zero_adv_val5':np.nan, 
        #         'dormant_adv_val5':np.nan,  
        #         'overactive_adv_val5':np.nan,   }
    
    dataframe.append(result)

    # 'gradient_norm5':gradient_norm,
    # 'gradient_norm0':gradient_norm,

Backbone: robust_convnext_tiny
Dataset: dtd
Loss: TRADES_v2
Backbone: convnext_tiny.fb_in22k
Dataset: dtd
Loss: TRADES_v2
Backbone: deit_small_patch16_224.fb_in1k
Dataset: dtd
Loss: TRADES_v2
Backbone: robust_deit_small_patch16_224
Dataset: dtd
Loss: TRADES_v2
Backbone: vit_base_patch16_224.sam_in1k
Dataset: dtd
Loss: TRADES_v2
Backbone: convnext_base
Dataset: dtd
Loss: TRADES_v2
Backbone: robust_convnext_base
Dataset: dtd
Loss: TRADES_v2
Backbone: convnext_base.fb_in22k
Dataset: dtd
Loss: TRADES_v2
Backbone: convnext_base.clip_laion2b
Dataset: dtd
Loss: TRADES_v2
Backbone: convnext_base.clip_laion2b_augreg
Dataset: dtd
Loss: TRADES_v2
Backbone: vit_base_patch16_224.augreg_in1k
Dataset: dtd
Loss: TRADES_v2
Backbone: vit_base_patch16_224.mae
Dataset: dtd
Loss: TRADES_v2
Backbone: vit_base_patch16_224.orig_in21k
Dataset: dtd
Loss: TRADES_v2
Backbone: vit_base_patch16_224.augreg_in21k
Dataset: dtd
Loss: TRADES_v2
Backbone: vit_base_patch16_224.dino
Dataset: dtd
Loss: TRADES_v2
Backbone: v

In [25]:
import pandas as pd
from scipy.stats import pearsonr, spearmanr

df2 = pd.DataFrame(dataframe)

df_merged = pd.merge(df, df2, on=["backbone", "loss_function", "dataset"], how="inner")

df_merged = df_merged.dropna(subset=["clean_acc", "robust_acc"])

df_merged.to_csv('results_corr.csv')

# Define a function to calculate correlation and p-values for a set of columns
def compute_correlations_with_pvalues(df, cols, target_vars):
    results = []
    
    for target in target_vars:  # clean_acc and robust_acc
        for col in cols:
            # Compute Pearson correlation and p-value

            # print( df[target], df[col] )
            pearson_corr, pearson_p = pearsonr(df[target], df[col])
            # Compute Spearman correlation and p-value
            spearman_corr, spearman_p = spearmanr(df[target], df[col])
            
            results.append({
                "target": target,
                "feature": col,
                "pearson_corr": pearson_corr,
                "pearson_pval": pearson_p,
                "spearman_corr": spearman_corr,
                "spearman_pval": spearman_p
            })
    
    return pd.DataFrame(results)

# Compute correlations and p-values for the specified columns
cols = [
    "zero_adv_train0",
    "dormant_adv_train0",
    "overactive_adv_train0",
    "zero_adv_train5",
    "dormant_adv_train5",
    "overactive_adv_train5",
    "zero_adv_val0",
    "dormant_adv_val0",
    "overactive_adv_val0",
]

target_vars = ["clean_acc", "robust_acc"]

# Correlation with p-values for the entire dataset
correlation_pvalues = compute_correlations_with_pvalues(df_merged, cols, target_vars)
# Set display option
pd.set_option("display.float_format", "{:.6f}".format)

correlation_pvalues

,target,feature,pearson_corr,pearson_pval,spearman_corr,spearman_pval
0,clean_acc,zero_adv_train0,0.026527,0.666703,-0.027356,0.656940
1,clean_acc,dormant_adv_train0,-0.226175,0.000199,-0.218891,0.000322
2,clean_acc,overactive_adv_train0,-0.279848,0.000004,-0.257647,0.000021
3,clean_acc,zero_adv_train5,-0.096493,0.116412,-0.084752,0.168129
4,clean_acc,dormant_adv_train5,-0.303157,0.000000,-0.264762,0.000012
5,clean_acc,overactive_adv_train5,-0.276310,0.000005,-0.274725,0.000005
6,clean_acc,zero_adv_val0,-0.152332,0.012872,-0.084616,0.168817
7,clean_acc,dormant_adv_val0,-0.362161,0.000000,-0.279111,0.000004
8,clean_acc,overactive_adv_val0,-0.295751,0.000001,-0.255786,0.000024
9,robust_acc,zero_adv_train0,-0.003403,0.955949,-0.049542,0.420995


In [26]:
from scipy.stats import pearsonr, spearmanr
import pandas as pd

# Define the column groups for analysis
cols = [
    "zero_adv_train0",
    "dormant_adv_train0",
    "overactive_adv_train0",
    "zero_adv_train5",
    "dormant_adv_train5",
    "overactive_adv_train5",
    "zero_adv_val0",
    "dormant_adv_val0",
    "overactive_adv_val0",
]

# Define the backbone and loss groupings
group_backbones = {
    "convnext": ['convnext_tiny', 'robust_convnext_tiny', 'convnext_tiny.fb_in22k',
                 'convnext_base', 'convnext_base.fb_in22k', 'robust_convnext_base',
                 'convnext_base.clip_laion2b', 'convnext_base.clip_laion2b_augreg',  ],

    "vit": ['deit_small_patch16_224.fb_in1k', 'robust_deit_small_patch16_224',
            'vit_base_patch16_224.augreg_in1k', 'vit_base_patch16_224.augreg_in21k',
            'vit_base_patch16_224.dino', 'vit_base_patch16_224.mae', 'vit_base_patch16_224.orig_in21k',
            'vit_base_patch16_224.sam_in1k', 'vit_base_patch16_224_miil.in21k' ]
}

group_losses = {
    "trades_v2": ["TRADES_v2"],
    "classic_at": ["CLASSIC_AT"]
}

# Function to compute Pearson and Spearman correlations
def compute_group_correlations_and_summary(df, group_filter, target_cols):
    subgroup = df[group_filter]
    correlations = []
    summary_stats = []

    for target_col in target_cols:
        for acc_col in ["clean_acc", "robust_acc"]:
            # Drop NaNs for valid correlation computation
            valid_data = subgroup[[acc_col, target_col]].dropna()

            # Compute correlations if enough data
            if len(valid_data) > 1:
                # Pearson correlation
                pearson_corr, pearson_pval = pearsonr(valid_data[acc_col], valid_data[target_col])
                # Spearman correlation
                spearman_corr, spearman_pval = spearmanr(valid_data[acc_col], valid_data[target_col])
            else:
                pearson_corr = pearson_pval = spearman_corr = spearman_pval = float("nan")

            correlations.append({
                "target": acc_col,
                "feature": target_col,
                "pearson_corr": pearson_corr,
                "pearson_pval": pearson_pval,
                "spearman_corr": spearman_corr,
                "spearman_pval": spearman_pval
            })



    return pd.DataFrame(correlations)

# Create group filters
group_filters = {
    "convnext_trades_v2": (df_merged["backbone"].isin(group_backbones["convnext"])) & (df_merged["loss_function"].isin(group_losses["trades_v2"])),
    "convnext_classic_at": (df_merged["backbone"].isin(group_backbones["convnext"])) & (df_merged["loss_function"].isin(group_losses["classic_at"])),
    "vit_trades_v2": (df_merged["backbone"].isin(group_backbones["vit"])) & (df_merged["loss_function"].isin(group_losses["trades_v2"])),
    "vit_classic_at": (df_merged["backbone"].isin(group_backbones["vit"])) & (df_merged["loss_function"].isin(group_losses["classic_at"]))
}

# Compute correlations and summary statistics for each group
group_results = {}
for group_name, group_filter in group_filters.items():
    correlations = compute_group_correlations_and_summary(df_merged, group_filter, cols)
    group_results[group_name] =  correlations
       




In [27]:
group_results['convnext_trades_v2']

,target,feature,pearson_corr,pearson_pval,spearman_corr,spearman_pval
0,clean_acc,zero_adv_train0,0.261253,0.026646,0.174679,0.142221
1,robust_acc,zero_adv_train0,0.171541,0.149641,0.118742,0.320492
2,clean_acc,dormant_adv_train0,-0.205041,0.084025,0.068908,0.565188
3,robust_acc,dormant_adv_train0,-0.006054,0.959745,0.099933,0.403597
4,clean_acc,overactive_adv_train0,-0.136995,0.251169,-0.080654,0.500634
5,robust_acc,overactive_adv_train0,0.022930,0.848380,-0.035048,0.770072
6,clean_acc,zero_adv_train5,0.352631,0.002381,0.266082,0.023873
7,robust_acc,zero_adv_train5,0.284550,0.015413,0.223485,0.059149
8,clean_acc,dormant_adv_train5,-0.009443,0.937252,0.107559,0.368488
9,robust_acc,dormant_adv_train5,0.033583,0.779440,0.062485,0.602065


In [28]:
group_results['convnext_classic_at']

,target,feature,pearson_corr,pearson_pval,spearman_corr,spearman_pval
0,clean_acc,zero_adv_train0,-0.116701,0.366368,-0.207489,0.105623
1,robust_acc,zero_adv_train0,-0.154738,0.229804,-0.143545,0.265689
2,clean_acc,dormant_adv_train0,0.009421,0.942065,0.012199,0.925024
3,robust_acc,dormant_adv_train0,0.187336,0.144838,0.142233,0.270128
4,clean_acc,overactive_adv_train0,-0.163512,0.204135,-0.198070,0.122782
5,robust_acc,overactive_adv_train0,0.009468,0.941777,-0.056378,0.663394
6,clean_acc,zero_adv_train5,-0.187899,0.143611,-0.275759,0.030053
7,robust_acc,zero_adv_train5,-0.170302,0.185715,-0.227681,0.075120
8,clean_acc,dormant_adv_train5,-0.198865,0.121257,-0.144042,0.264017
9,robust_acc,dormant_adv_train5,-0.151095,0.241093,-0.099140,0.443299


In [29]:
group_results['vit_trades_v2']

,target,feature,pearson_corr,pearson_pval,spearman_corr,spearman_pval
0,clean_acc,zero_adv_train0,0.031827,0.799734,-0.076089,0.543705
1,robust_acc,zero_adv_train0,0.062154,0.620055,0.018218,0.884564
2,clean_acc,dormant_adv_train0,-0.073308,0.558569,-0.158487,0.203726
3,robust_acc,dormant_adv_train0,0.033053,0.792195,-0.033406,0.790022
4,clean_acc,overactive_adv_train0,-0.336739,0.005698,-0.317748,0.009327
5,robust_acc,overactive_adv_train0,-0.275540,0.025138,-0.338031,0.005505
6,clean_acc,zero_adv_train5,-0.154568,0.215277,-0.183057,0.141235
7,robust_acc,zero_adv_train5,-0.108546,0.385639,-0.108742,0.384777
8,clean_acc,dormant_adv_train5,-0.383035,0.001502,-0.360075,0.002981
9,robust_acc,dormant_adv_train5,-0.226954,0.066870,-0.261357,0.034031


In [30]:
group_results['vit_classic_at']

,target,feature,pearson_corr,pearson_pval,spearman_corr,spearman_pval
0,clean_acc,zero_adv_train0,-0.101951,0.415327,-0.047629,0.704122
1,robust_acc,zero_adv_train0,-0.090184,0.471447,-0.064653,0.606031
2,clean_acc,dormant_adv_train0,-0.281241,0.022164,-0.273464,0.026302
3,robust_acc,dormant_adv_train0,-0.225296,0.068941,-0.218178,0.078427
4,clean_acc,overactive_adv_train0,-0.196297,0.114183,-0.104140,0.405330
5,robust_acc,overactive_adv_train0,-0.190594,0.125302,-0.168842,0.175347
6,clean_acc,zero_adv_train5,-0.380063,0.001646,-0.249807,0.043088
7,robust_acc,zero_adv_train5,-0.302484,0.013566,-0.271003,0.027740
8,clean_acc,dormant_adv_train5,-0.347249,0.004282,-0.275283,0.025280
9,robust_acc,dormant_adv_train5,-0.287637,0.019188,-0.242793,0.049498


In [31]:
import pandas as pd

# Function to compute summary statistics for clean_acc and robust_acc
def compute_summary_statistics(df, group_filter):
    subgroup = df[group_filter]
    summary_stats = []

    for acc_col in ["clean_acc", "robust_acc"]:
        # Compute statistics
        summary_stats.append({
            "metric": acc_col,
            "mean": subgroup[acc_col].mean(),
            "std_dev": subgroup[acc_col].std(),
            "min": subgroup[acc_col].min(),
            "max": subgroup[acc_col].max(),
            "count": subgroup[acc_col].count()
        })

    return pd.DataFrame(summary_stats)

# Compute statistics for each group
group_stats = {}
for group_name, group_filter in group_filters.items():
    stats = compute_summary_statistics(df_merged, group_filter)
    group_stats[group_name] = stats

# Display results
for group_name, stats in group_stats.items():
    print(f"\nGroup: {group_name}")
    print(stats)


Group: convnext_trades_v2
       metric     mean  std_dev      min      max  count
0   clean_acc 0.667398 0.258302 0.075333 0.982000     72
1  robust_acc 0.394134 0.266194 0.000667 0.931333     72

Group: convnext_classic_at
       metric     mean  std_dev      min      max  count
0   clean_acc 0.585448 0.292469 0.063333 0.983333     62
1  robust_acc 0.363318 0.297786 0.000000 0.929333     62

Group: vit_trades_v2
       metric     mean  std_dev      min      max  count
0   clean_acc 0.563162 0.303483 0.010000 0.960000     66
1  robust_acc 0.291038 0.272680 0.002667 0.890000     66

Group: vit_classic_at
       metric     mean  std_dev      min      max  count
0   clean_acc 0.472909 0.302407 0.006667 0.962000     66
1  robust_acc 0.313773 0.276086 0.000000 0.904667     66
